In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test  = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [ ]:
train.columns[train.isnull().any()]

In [ ]:
train.Age = train.Age.fillna(train.Age.mean())
test.Age  = test.Age.fillna(test.Age.mean())

In [ ]:
train.Ticket = train.Ticket.fillna('Missing')
test.Ticket  = test.Ticket.fillna('Missing')

In [ ]:
train.Fare = train.Fare.fillna(train.Fare.mean())
test.Fare  = test.Fare.fillna(test.Fare.mean())

In [ ]:
train.Cabin = train.Cabin.fillna('Missing')
test.Cabin  = test.Cabin.fillna('Missing')

In [ ]:
train.Embarked = train.Embarked.fillna('M')
test.Embarked  = test.Embarked.fillna('M')

In [ ]:
train.Sex = train.Sex.astype('category')
train.Sex = train.Sex.cat.codes
test.Sex = test.Sex.astype('category')
test.Sex = test.Sex.cat.codes

In [ ]:
train.Embarked = train.Embarked.astype('category')
train.Embarked = train.Embarked.cat.codes
test.Embarked = test.Embarked.astype('category')
test.Embarked = test.Embarked.cat.codes

In [ ]:
train.Cabin = train.Cabin.astype('category')
train.Cabin = train.Cabin.cat.codes
test.Cabin = test.Cabin.astype('category')
test.Cabin = test.Cabin.cat.codes

In [ ]:
# train

In [ ]:
# import seaborn as sns
# import numpy as np

# sns.heatmap(train.corr(), mask=np.triu(train.corr()) + np.eye(len(train.corr())))

In [ ]:
nn_train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
nn_test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X_test = nn_test

In [ ]:
data   = nn_train
labels = nn_train['Survived']
data = data.drop(['Survived'], axis=1)

In [ ]:
# data

In [ ]:
X_train = data[:90000]
y_train = labels[:90000]

X_valid = data[90000:]
y_valid = labels[90000:]

In [ ]:
# X_train.shape

In [ ]:
# import tensorflow as tf

In [ ]:
# # model = tf.keras.Sequential([
# #             tf.keras.layers.Dense(128, activation='relu'),
# #             tf.keras.layers.Dense(256, activation='relu'),
# # #             tf.keras.layers.Dropout(0.6),
# #             tf.keras.layers.Dense(2, activation='softmax')                             
# #         ])

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv1D(32, 2, activation='relu', input_shape=(None, 8)),
#     tf.keras.layers.Conv1D(32, 2, activation='relu'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(2, activation='softmax')
# ])

# model.summary()

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=3e-4), metrics=['accuracy'])

In [ ]:
# np.expand_dims(X_train, axis=1).shape

In [ ]:
# import numpy as np

# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./model.ckpt', \
#                                                  monitor='val_accuracy', save_weights_only=True, save_best_only=True, \
#                                                  verbose=1)

# # model.fit(X_train, tf.one_hot(np.array(y_train), depth=2), epochs=10000, batch_size=1000, \
# #           validation_data=(X_valid, tf.one_hot(np.array(y_valid), depth=2)), callbacks=[cp_callback])

# model.fit(np.expand_dims(X_train, axis=1), tf.one_hot(np.array(y_train), depth=2), epochs=10000, batch_size=1000, \
#           callbacks=[cp_callback])

In [ ]:
# model.predict(np.array([X_test.loc[0, :]]))

In [ ]:
# X_train.dtypes

In [ ]:
from catboost import CatBoostClassifier, Pool, cv

# 'border_count': 105.0,
#  'depth': 7.0,
#  'iterations': 950.0,
#  'l2_leaf_reg': 3.0,
#  'learning_rate': 0.15135587693365354,
#  'one_hot': 53.0

model = CatBoostClassifier(
      custom_loss=['Accuracy'],
      random_seed=42,
      logging_level='Silent',
      # task_type='GPU'
      learning_rate = 0.15135587693365354,
      depth=7.0,
      l2_leaf_reg = 3.0,
      border_count = 105,
      iterations=950,
      one_hot_max_size = 53
    )

In [ ]:
model.fit(
      X_train, y_train,
      eval_set=(X_valid, y_valid),
#       logging_level='Verbose',  # you can uncomment this for text output
      plot=True
    
    )

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score

y_pred = model.predict_proba(X_valid)[:, 1]
valid_score = accuracy_score(y_valid, np.where(y_pred > 0.56, 1, 0))
print(valid_score)

In [ ]:
test_pred = np.where(model.predict_proba(X_test)[:, 1] > 0.55, 1, 0)

In [ ]:
X_train

In [ ]:
# X_test

In [ ]:
df = pd.DataFrame(data={'PassengerId': pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")['PassengerId'],
                       'Survived': test_pred})

In [ ]:
# df

In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
# pd.read_csv('./submission.csv')

In [ ]:
# from hyperopt import hp, tpe, Trials, STATUS_OK
# from hyperopt import fmin

In [ ]:
# def org_results(trials, hyperparams, model_name):
#     train_time = str(trials[-1]['refresh_time'] - trials[0]['book_time'])
#     acc = round(trials[fit_idx]['result']['accuracy'], 3)
#     train_auc = round(trials[fit_idx]['result']['train auc'], 3)
#     test_auc = round(trials[fit_idx]['result']['test auc'], 3)

#     results = {
#         'model': model_name,
#         'parameter search time': train_time,
#         'accuracy': acc,
#         'test auc score': test_auc,
#         'training auc score': train_auc,
#         'parameters': hyperparams
#     }
#     return results

In [ ]:
# def cat_objective(space):
    
#     # cboost = CatBoostClassifier(
#     #   eval_metric  = 'AUC',
#     #   learning_rate = space['learning_rate'],
#     #   iterations = space['iterations'],
#     #   depth = space['depth'],
#     #   l2_leaf_reg = space['l2_leaf_reg'],
#     #   border_count = space['border_count']
#     #   task_type='GPU'
#     # )
    
#     # cboost = CatBoostClassifier(
#     #     custom_loss=['Accuracy'],
#     #     random_seed=42,
#     #     logging_level='Silent',
#     #     one_hot_max_size=space['one_hot'],
#     #     task_type='GPU'
#     # )

#     model.fit(X_train, y_train, 
#               eval_set = (X_valid, y_valid), 
#               verbose = False)
    
#     predictions = model.predict(X_valid)
#     test_preds = model.predict_proba(X_valid)[:,1]
#     train_preds = model.predict_proba(X_train)[:,1]    

#     train_auc = roc_auc_score(y_train, train_preds)
#     test_auc = roc_auc_score(y_valid, test_preds)
#     accuracy = accuracy_score(y_valid, predictions)
    
#     return {'status': STATUS_OK, 'loss': 1-test_auc, 'accuracy': accuracy,
#             'test auc': test_auc, 'train auc': train_auc}
    
# trials = Trials()
# space = {
#     'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.3)),
#     'iterations': hp.quniform('iterations', 25, 1000, 25),
#     'depth': hp.quniform('depth', 1, 16, 1),
#     'border_count': hp.quniform('border_count', 30, 220, 5), 
#     'l2_leaf_reg': hp.quniform('l2_leaf_reg', 1, 10, 1),
#     'one_hot': hp.quniform('one_hot', 2, 256, 1)
# }

# cboost_hyperparams = fmin(fn = cat_objective, 
#                  max_evals = 150, 
#                  trials = trials,
#                  algo = tpe.suggest,
#                  space = space)

# cbo_results = org_results(trials.trials, cboost_hyperparams, 'CatBoost')
# display(cbo_results)

In [ ]:
# cboost_hyperparams